# <추천시스템 1강>

목표 : 어떤 사용자에게 어떤 상품을 어떻게 추천할지에 대해 이해
-기업에서 추천시스템
당근마켓 : 다른 사람들이 같이 본 상품추천
카카오 브런치 : 해당 글과 유사한 글을 추천


2010 apriori                     

2015 협업필터링,svd    

2013~ 2017 spark사용 FP-Growth, Matrix Factoriztion

2015~2017 딥러닝+협업필터링, item2vec, doc2vec, wide & deep model          

2017~ 개인화  factoriztion machine, hierarchical rnn, 강화학습+re-ranking, 딥러닝

<연관분석(association analysis)>

연관-> 

1. 얼머나 같이 구매되는가? 

2. a아이템을 구매하는 사람이 b아이템을 구매하는가

ex) 맥주, 기저귀

지지도 a->b : p(a)
향상도 a->b : p(a,b)/(p(a)*p(b))  동시에 얼마나 발생하는지
신뢰도 a->b :  p(a,b)/p(a)      a일어날때 b발생

규칙생성 -> 모든 경우의 수(아이템 커지면 너무 많아짐)를 탐색해 지지도. 신뢰도, 향상도 높은 규칙 찾음
            -> 모든 경우의 수 고려 안하기위해 apriori(빈번하지 않은 것이 포함된 셋 제외)사용


apriori개선 -> FP-Growth

1. 모든거래에서 최소 지지도 이상의 아이템만 선택
2. 모든거래에서 빈도가 높은순으로 아이템 순서대로 순서를 정렬
3. 부모 노드를 중심으로 거래를 자식노드로 추가해주면서 tree생성
4. 새로운 아이템이 나올 경우에 부모노드로부터 시작하고, 그렇지 않으면 기존의 노드에서 확장
5. 지지도가 낮은 순서부터 시작하여 조건부 패턴을 생성
6. 모든 아이템에 대해서 반복
7. 조건부 패턴 기반으로 패턴 생성
-> www.kaggle.com/chocozzz/00-apriori-fp-growth 실습

In [3]:
import mlxtend
import numpy as np
import pandas as pd

In [4]:
data = np.array([ #-> sparse matrix로 바꿔줘야함 
    ['우유', '기저귀', '쥬스'],
    ['양상추', '기저귀', '맥주'],
    ['우유', '양상추', '기저귀', '맥주'],
    ['양상추', '맥주']
])

<ipython-input-4-d123d296d708>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  data = np.array([


# Apriori 알고리즘 

In [6]:
from mlxtend.preprocessing import TransactionEncoder # -> trasaction matrix로 변환해주는 패키지
te = TransactionEncoder()
te_ary = te.fit(data).transform(data)
df = pd.DataFrame(te_ary, columns=te.columns_)
df 

,기저귀,맥주,양상추,우유,쥬스
0,True,False,False,True,True
1,True,True,True,False,False
2,True,True,True,True,False
3,False,True,True,False,False


In [17]:

%%time 
# 시간확인

from mlxtend.frequent_patterns import apriori  # apriori 알고리즘 호출 

apriori(df, min_support=0.5, use_colnames=True) # min_support 로 최소 지지도 설정 가능

Wall time: 7.98 ms


,support,itemsets
0,0.75,(기저귀)
1,0.75,(맥주)
2,0.75,(양상추)
3,0.50,(우유)
4,0.50,"(기저귀, 맥주)"
5,0.50,"(기저귀, 양상추)"
6,0.50,"(기저귀, 우유)"
7,0.75,"(양상추, 맥주)"
8,0.50,"(기저귀, 양상추, 맥주)"


### 연산량이 많고 많은 연관상품이 나타나는 단점

# FP-Growth

In [18]:
te = TransactionEncoder()
te_ary = te.fit(data).transform(data)
df = pd.DataFrame(te_ary, columns=te.columns_)
df

,기저귀,맥주,양상추,우유,쥬스
0,True,False,False,True,True
1,True,True,True,False,False
2,True,True,True,True,False
3,False,True,True,False,False


In [19]:
%%time

from mlxtend.frequent_patterns import fpgrowth

fpgrowth(df, min_support=0.5, use_colnames=True)

# 속도가 apriori보다 fp-growth보다 3배정도 빠름

Wall time: 2.99 ms


,support,itemsets
0,0.75,(기저귀)
1,0.50,(우유)
2,0.75,(양상추)
3,0.75,(맥주)
4,0.50,"(기저귀, 맥주)"
5,0.50,"(기저귀, 양상추)"
6,0.50,"(기저귀, 양상추, 맥주)"
7,0.50,"(기저귀, 우유)"
8,0.75,"(양상추, 맥주)"


### 대용량 데이터에서 메모리 효율 낮음, 지지도 계산이 FP-tree만들어지고 나서 가능하다는 단점